In [1]:
from sqlalchemy import create_engine
import pandas as pd
import psycopg2 
import io

from pgspecial.main import PGSpecial

In [2]:
# If you turn this feature on, you can display each 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In-line SQL

In [21]:
%load_ext sql
%load_ext pgcli.magic

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [58]:
%%sql postgresql://weaves:3lbpb84@j1/weaves
select count(*) from actor

1 rows affected.


count
200


In [5]:
version = %sql select version()

 * postgresql://weaves:***@j1/weaves
1 rows affected.


In [6]:
version

version
"PostgreSQL 11.5 (Debian 11.5-1+deb10u1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit"


In [7]:
engine = create_engine('postgresql+psycopg2://weaves:3lbpb84@j1:5432/weaves')

In [8]:
ntbl0 = 'users'
ftbl0 = 'cache/in/users.csv2'

In [9]:
# I'm going to user my simplified CSV tables
# And create tables on my remote erver 
tbl1 = pd.read_csv(ftbl0, index_col=0, nrows=10)
tbl1.info()
tbl1.columns
tbl1.columns = ["userid","ctry","city","dt0","uscrypt0","plan","push1","email1","ncontacts","nrefls","nsrefls"]
tbl1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 11 columns):
birth_year                                  10 non-null int64
country                                     10 non-null object
city                                        0 non-null float64
created_date                                10 non-null object
user_settings_crypto_unlocked               10 non-null int64
plan                                        10 non-null object
attributes_notifications_marketing_push     4 non-null float64
attributes_notifications_marketing_email    4 non-null float64
num_contacts                                10 non-null int64
num_referrals                               10 non-null int64
num_successful_referrals                    10 non-null int64
dtypes: float64(3), int64(5), object(3)
memory usage: 960.0+ bytes


Index(['birth_year', 'country', 'city', 'created_date',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 11 columns):
userid       10 non-null int64
ctry         10 non-null object
city         0 non-null float64
dt0          10 non-null object
uscrypt0     10 non-null int64
plan         10 non-null object
push1        4 non-null float64
email1       4 non-null float64
ncontacts    10 non-null int64
nrefls       10 non-null int64
nsrefls      10 non-null int64
dtypes: float64(3), int64(5), object(3)
memory usage: 960.0+ bytes


In [10]:
tbl1['dt0'] = pd.to_datetime(tbl1['dt0'])
tbl1[tbl1.select_dtypes(['object']).columns] = tbl1.select_dtypes(['object']).apply(lambda x: x.astype('category'))
tbl1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 0 to 9
Data columns (total 11 columns):
userid       10 non-null int64
ctry         10 non-null category
city         0 non-null float64
dt0          10 non-null datetime64[ns]
uscrypt0     10 non-null int64
plan         10 non-null category
push1        4 non-null float64
email1       4 non-null float64
ncontacts    10 non-null int64
nrefls       10 non-null int64
nsrefls      10 non-null int64
dtypes: category(2), datetime64[ns](1), float64(3), int64(5)
memory usage: 1.1 KB


In [11]:
# More types can be forced at the server using SQL Alchemy.
# Pandas will build instruct that indices are built on each column for you - see later.
tbl1.to_sql(ntbl0, engine, if_exists='replace', chunksize=4000, method='multi')

In [12]:
tbl0 = pd.read_sql_query("SELECT * FROM %s;" % ntbl0, engine)
tbl0

,user_id,userid,ctry,city,dt0,uscrypt0,plan,push1,email1,ncontacts,nrefls,nsrefls
0,0,1989,PL,None,2018-01-13 05:15:15.599466,1,STANDARD,1.0,1.0,3,0,0
1,1,1975,GB,None,2018-01-29 03:38:46.676876,0,STANDARD,NaN,NaN,21,0,0
2,2,1987,PL,None,2018-01-18 19:17:31.229096,0,STANDARD,0.0,0.0,21,0,0
3,3,1994,FR,None,2018-01-15 18:47:56.723104,0,STANDARD,1.0,0.0,0,0,0
4,4,1985,GB,None,2018-01-11 00:36:46.673673,0,STANDARD,NaN,NaN,2,0,0
5,5,1993,LT,None,2018-01-04 17:45:03.907804,1,STANDARD,NaN,NaN,0,0,0
6,6,1978,GB,None,2018-01-24 19:04:50.876183,0,STANDARD,NaN,NaN,0,0,0
7,7,1989,RO,None,2018-01-11 19:11:45.757486,0,STANDARD,1.0,1.0,0,0,0
8,8,1981,GB,None,2018-01-03 07:28:59.063740,1,STANDARD,NaN,NaN,0,0,0
9,9,1965,PL,None,2018-01-09 10:21:19.452972,1,PREMIUM,NaN,NaN,32,0,0


In [13]:
%sql select * from information_schema.columns where table_name = 'users';

 * postgresql://weaves:***@j1/weaves
12 rows affected.


table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
weaves,public,users,user_id,1,None,YES,bigint,None,None,64,2,0,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,int8,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,users,userid,2,None,YES,bigint,None,None,64,2,0,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,int8,None,None,None,None,2,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,users,ctry,3,None,YES,text,None,1073741824,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,text,None,None,None,None,3,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,users,city,4,None,YES,double precision,None,None,53,2,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,float8,None,None,None,None,4,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,users,dt0,5,None,YES,timestamp without time zone,None,None,None,None,None,6,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,timestamp,None,None,None,None,5,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,users,uscrypt0,6,None,YES,bigint,None,None,64,2,0,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,int8,None,None,None,None,6,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,users,plan,7,None,YES,text,None,1073741824,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,text,None,None,None,None,7,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,users,push1,8,None,YES,double precision,None,None,53,2,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,float8,None,None,None,None,8,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,users,email1,9,None,YES,double precision,None,None,53,2,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,float8,None,None,None,None,9,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,users,ncontacts,10,None,YES,bigint,None,None,64,2,0,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,int8,None,None,None,None,10,NO,NO,None,None,None,None,None,NO,NEVER,None,YES


In [14]:
# It's also possible to use PGSpecial to use the Postgres special commands.

In [15]:
# See some meta-information in IPython
p = PGSpecial()
conn = psycopg2.connect(user='weaves', host='j1', database='weaves', password='3lbpb84')

In [57]:
%sql truncate table users

 * postgresql://weaves:***@j1/weaves
(psycopg2.errors.QueryCanceled) canceling statement due to user request

[SQL: truncate table users]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [55]:
%sql select count(*) from users

 * postgresql://weaves:***@j1/weaves
1 rows affected.


count
19430


In [56]:
cptbl0 = "\\copy users from '{}' with delimiter ',' CSV HEADER".format(ftbl0)
cptbl0

"\\copy users from 'cache/in/users.csv2' with delimiter ',' CSV HEADER"

In [39]:
cur = conn.cursor()

p.execute(cur, cptbl0)

[(None, None, None, '')]

In [53]:
%sql select count(*) from users

 * postgresql://weaves:***@j1/weaves
1 rows affected.


count
19430


In [51]:
## PGSpecial doesn't seem to want to do this
## So set-up .pgpass and use a direct invocation using the shell.

from IPython.display import display_html

host='j1'
database='weaves'
username='weaves'

#psql_cmd=raw_input("Enter psql command to execute: ")
#psql_cmd='\l'
psql_cmd=u'\copy users from \'{}\' WITH DELIMITER \',\' CSV HEADER'.format(ftbl0)
psql_cmd
# psql_cmd="\d users"

"\\copy users from 'cache/in/users.csv2' WITH DELIMITER ',' CSV HEADER"

In [52]:
psql_out = !psql -H -h {host} -d {database} -U {username} -c "{psql_cmd}"
display_html("".join(psql_out), raw=True)

COPY 19430

In [37]:
# This gets the meta-data for a table
cur = conn.cursor()

for title, rows, headers, status in p.execute(cur, '\\d users'):
    print(title)
    print(headers)
    for row in rows:
        print(row)
        print(status)

None
['Column', 'Type', 'Modifiers']
['user_id', 'bigint', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['userid', 'bigint', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['ctry', 'text', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['city', 'double precision', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['dt0', 'timestamp without time zone', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['uscrypt0', 'bigint', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['plan', 'text', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['push1', 'double precision', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['email1', 'double precision', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['ncontacts', 'bigint', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['nrefls', 'bigint', '']
Indexes:
    "ix_users_user_id" btree (user_id)

['nsrefls', 'bigint', '']
Indexes:
    "ix_users_user_id" btree (user_id)



## q/kdb+

To get 10 rows of a table send a q command. To get the meta-data of a table, use the meta command

In [42]:
import urllib.parse

## Getting tables from a q/kdb+ server need a URL quote if you just want 10 lines. 10# means take 10.
s0 = urllib.parse.quote('10#users0')
s0
url0="http://j1:4444/q.csv?{}".format(s0)
c = pd.read_csv(url0)
c

'10%23users0'

,userid,yob,ctry,dt0,uscrypt0,plan,push1,email1,ncontacts,rafu
0,0,1989,PL,2018-01-13 05:15:15.599,1,STANDARD,1.0,1.0,3,1
1,1,1975,GB,2018-01-29 03:38:46.676,0,STANDARD,NaN,NaN,21,0
2,2,1987,PL,2018-01-18 19:17:31.229,0,STANDARD,0.0,0.0,21,0
3,3,1994,FR,2018-01-15 18:47:56.723,0,STANDARD,1.0,0.0,0,0
4,4,1985,GB,2018-01-11 00:36:46.673,0,STANDARD,NaN,NaN,2,0
5,5,1993,LT,2018-01-04 17:45:03.907,1,STANDARD,NaN,NaN,0,1
6,6,1978,GB,2018-01-24 19:04:50.876,0,STANDARD,NaN,NaN,0,0
7,7,1989,RO,2018-01-11 19:11:45.757,0,STANDARD,1.0,1.0,0,0
8,8,1981,GB,2018-01-03 07:28:59.063,1,STANDARD,NaN,NaN,0,0
9,9,1965,PL,2018-01-09 10:21:19.452,1,PREMIUM,NaN,NaN,32,1


In [44]:
# Meta information
s0 = urllib.parse.quote('meta users0')
s0
url0="http://j1:4444/q.csv?{}".format(s0)
c = pd.read_csv(url0)
c

'meta%20users0'

,c,t,f,a
0,userid,i,NaN,s
1,yob,h,NaN,NaN
2,ctry,s,NaN,NaN
3,dt0,z,NaN,NaN
4,uscrypt0,b,NaN,NaN
5,plan,s,NaN,NaN
6,push1,e,NaN,NaN
7,email1,e,NaN,NaN
8,ncontacts,i,NaN,NaN
9,rafu,b,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
c    10 non-null object
t    10 non-null object
f    0 non-null float64
a    1 non-null object
dtypes: float64(1), object(3)
memory usage: 400.0+ bytes
